In [3]:
# importing the necessary packages
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET # for the xml files
import pickle # for compression of files
import math
from tqdm import tqdm # for the progress bar
import re # for regex


# setting the main file path to where all files are
os.chdir("C:\\Users\\SJNic\\Documents\\University\\Master\\Thesis\\Thesis Code")

# setting the directory containing the meetings
meetings_dir = "20210415/"

# Retrieve all meeting files

In [4]:
# retrieve all meeting files
meetings = [s for s in os.listdir(meetings_dir) if "meetings_" in s]

In [50]:
# dictionary version
# first create a list containing all meetings and their corresponding information
# list containing information about meetings
meetingsInformation = {}

# list containing all the item information
itemsInformation = {}

# list containing all the document information
documentsInformation = {}

for file in meetings:
    
    # reading only the second to last xml file
    tree = ET.parse(meetings_dir + file)
    root = tree.getroot()

    # loop through all meetings
    for meeting in root:
        #print(meeting.tag)

        # intermediary information for the meetings
        meetingInformation = {}


        meetingInformation["meeting id"] = meeting.attrib["id"]
        
        # list to store the items in
        items = []
        # Go through all children of the meeting
        for child in meeting:
            if child.text == "\n      " and not child.tag == "item":
                meetingInformation[child.tag] = child.attrib
            elif not child.tag == "item":
                meetingInformation[child.tag] = child.text





            # if child is an item go through all the children of the item and store their information
            if child.tag == "item":
                
                # append item to items
                items.append(child.attrib)
                
                # intermediary information for the items
                itemInformation = {}
                itemInformation["item id"] = child.attrib["id"]
                
                # list for saving all documents attached to this item
                documents = []
                
                # loop through all items
                for itemChild in child:
                    
                    
                    if itemChild.text == "\n        " and not child.tag == "document":
                        #print(itemChild.text)
                        itemInformation[itemChild.tag] = itemChild.attrib
                    elif not child.tag == "document":
                        itemInformation[itemChild.tag] = itemChild.text

                        
                        
                        
                    # now repeat once more to find the documents
                    if itemChild.tag == "document":
                        
                        # append this document to list of documents
                        documents.append(itemChild.attrib)
                        
                        # intermediary information for the document
                        documentInformation = {}
                        documentInformation["document id"] = itemChild.attrib["id"]

                        for documentChild in itemChild:
                            if itemChild.text == "\n     ":
                                documentInformation[documentChild.tag] = documentChild.attrib
                            else :
                                documentInformation[documentChild.tag] = documentChild.text


                        # store all information for this document
                        documentsInformation[itemChild.attrib["id"]] =  documentInformation

                    # attach all documents belonging to this item
                    itemInformation["Documents"] = documents
                # store all information for this item 
                itemsInformation[child.attrib["id"]] = itemInformation
            
            # save the items attached to this meeting
            meetingInformation["items"] = items
         # store all information for this meeting
        meetingsInformation[meeting.attrib["id"]] = meetingInformation

# Reading the list files
First we will retrieve every document from the lists,
and then we will try to connect at least the moties and amendementen to some of the items

In [21]:
# retrieve all meeting files
lists = [s for s in os.listdir(meetings_dir) if "list_" in s]

# dict with the information about the documents in the lists
list_docs = {}

for file in lists:
    
    # reading only the second to last xml file
    tree = ET.parse(meetings_dir + file)
    root = tree.getroot()
    
    # loop through all entries
    for entry in root:
        
        # try to find the document
        for child in entry:
            if child.tag == "document":
                
                # intermediary information for the document
                documentInformation = {}
                documentInformation["document id"] = child.attrib

                for documentChild in child:
                    if documentChild.text == "\n     ":
                        documentInformation[documentChild.tag] = documentChild.attrib
                    else :
                        documentInformation[documentChild.tag] = documentChild.text
                
                # store all information for this document
                #list_docs[child.attrib["id"]] =  documentInformation
                documentsInformation[child.attrib["id"]] =  documentInformation

In [7]:
len(documentsInformation)

52137

## Adding Motie & amendementen links from list to meetings
Sometimes no match between an agendapoint and an item could be found, but this number was lower than 30.

By adding the moties & amendementen links, we went from 37279 documents attached to items to 40706.

Also went from 36356 item matches and a max of 13 items per document to the documents to 40706 and 13 items per document.
Without the M&A this number is 37279 with only a max of 10 links per document, but that was without duplicate detection

In [23]:
# function for finding an item based on the name and feature
def find_item(itemName, feature, itemsInformation, print_bool = False):
    for key in itemsInformation:
        item = itemsInformation[key]
        
        if itemName in item["title"] and feature == item["features"]:
            
            for child in item:
                if print_bool:
                    print(f"{child}: \t\t{item[child]}")
            if print_bool:
                print("_________________________________________")
            return item["item id"]

    #print("No item related to this name could be found")
    return ""

In [24]:
MA_lists = ["list_b25f250f-d487-4ce5-93b3-efb84d24567a.xml", "list_50ba9ec6-f591-4070-8eea-4755d4189973.xml"]

In [25]:
# dict with the information about the documents in the lists
list_docs = {}

# regex string for finding the specific agenda point text
ag = re.compile("\W[n]\d{1,4}.{0,3}\s+(.+)")
# regex string for finding the feature of the item
fe = re.compile("\W[n](\d{1,4}.{0,3})\s")

break2 = False
for file in MA_lists:
    
    # reading only the second to last xml file
    tree = ET.parse(meetings_dir + file)
    root = tree.getroot()
    
    # loop through all entries
    for entry in tqdm(root):
        
        docID = ""
        # try to find the document
        for child in entry:
            
            if child.tag == "document":
                docID = child.attrib["id"]
            
            attrib = child.attrib
            # check if this property is an agenda point en if it is not empty
            if list(attrib.values())[0] == "Agendapunt" and child.text != None:
                itemName = ag.findall(child.text)[0]
                feature = fe.findall(child.text)[0]
                
                # find the item that matches the name and feature
                itemID = find_item(itemName, feature, itemsInformation)
                
                # if an item was found then append this docID to that item's list of documents
                if itemID != "":
                    itemsInformation[itemID]["Documents"].append({"id": docID})
                

100%|██████████| 1802/1802 [00:04<00:00, 428.47it/s]


In [26]:
# count the number of docs attached to items
total_docs = 0
for key in itemsInformation:
    item = itemsInformation[key]
    total_docs += len(itemsInformation[key]["Documents"])
total_docs

40706

In [65]:
len()

100%|██████████| 52137/52137 [19:31<00:00, 44.49it/s]


447089

# Duplicate detection
Looking for a range of values in stead of an exact match leads to finding false positive duplicates.

It is only necessary to look for whether the filename or displayname of docB is in the filename of docA. Also looking for the same displayname of docA leads to no further connections, because the dictionaries were identical.
remark on that last sentence, it seems that this might not be fully true. Looks like about 2000 documents are not found then.

In [157]:
for doc in documentsInformation:
    if "Voorstel_12315" in documentsInformation[doc]["filename"]:
        print(documentsInformation[doc])

{'document id': '14a2161b-6f93-452d-945e-66bd2b0701c6', 'displayname': 'Voorstel_12315.pdf', 'filename': 'Voorstel_12315.pdf', 'filesize': '421943', 'publicdownloadurl': 'https://api1.ibabs.eu/publicdownload.aspx?site=Utrecht&id=14a2161b-6f93-452d-945e-66bd2b0701c6'}
{'document id': '3976546a-58c6-42d3-96af-7e0643038b85', 'displayname': 'Voorstel_12315.pdf', 'filename': 'Voorstel_12315.pdf', 'filesize': '421943', 'publicdownloadurl': 'https://api1.ibabs.eu/publicdownload.aspx?site=Utrecht&id=3976546a-58c6-42d3-96af-7e0643038b85'}
{'document id': '5c7abeb6-41f1-4875-8e70-7e0f4fbc5622', 'displayname': 'Voorstel_12315', 'filename': 'Voorstel_12315.pdf', 'filesize': '421943', 'publicdownloadurl': 'https://api1.ibabs.eu/publicdownload.aspx?site=Utrecht&id=5c7abeb6-41f1-4875-8e70-7e0f4fbc5622'}


#### Creating a reference to the "unique" ID

In [28]:
def find_document_references(docA, references, distance_percentage):    
    
    for docBID in documentsInformation:    
        docB = documentsInformation[docBID]


        # find all files where the filename or displayname of the second doc is in either de displayName or filename of the original doc
        if (docB["displayname"] in docA["filename"] or docB["filename"] in docA["filename"]): #or (docB["displayname"] in docA["displayname"] or docB["filename"] in docA["displayname"]):
            
            # checking for this makes the code a bit faster
            if distance_percentage == 0:
                # then only do something if the filesize is the same value
                if float(docA["filesize"]) == float(docB["filesize"]):
                    references[docA["filename"]+docA["filesize"]] = docB["document id"]
            
            # otherwise check for a range of possible values
            elif distance_percentage != 0:
                # then only do something if the filesize is around the same value
                if float(docA["filesize"]) * (1-distance_percentage) <= float(docB["filesize"]) <= float(docA["filesize"]) * (1+distance_percentage):
                    references[docA["filename"]+docA["filesize"]] = docB["document id"]
                    #print(docB)
    
    return references

In [29]:
document_references = {}

# loop through all documents and find the reference document for that document
for key in tqdm(documentsInformation):
    doc = documentsInformation[key]
    
    document_references = find_document_references(doc, document_references, 0)

  0%|          | 143/52137 [00:04<27:50, 31.13it/s]


KeyboardInterrupt: 

In [56]:
unique_IDs = []

for key in tqdm(list(document_references.values())):
    if key not in unique_IDs:
        unique_IDs.append(key)

100%|██████████| 49145/49145 [00:43<00:00, 1139.25it/s] 


In [58]:
print(len(documentsInformation))
len(unique_IDs)

52137


48935

In [64]:
# save the referebces
#with open("NewProcessingData/document_references.txt", "wb") as fp:
#    pickle.dump(document_references, fp)
# save the unique_IDs
with open("NewProcessingData/unique_IDs.txt", "wb") as fp:
    pickle.dump(unique_IDs, fp)

In [30]:
# load the references
with open("NewProcessingData/document_references.txt", "rb") as fp:   
    document_references = pickle.load(fp)
# load the unique_IDs
with open("NewProcessingData/unique_IDs.txt", "rb") as fp:
    unique_IDs = pickle.load(fp)

In [13]:
len(document_references)

49145

# Updating documentsInformation to have a reference to the unique docID

In [73]:
for key in tqdm(documentsInformation):
    doc = documentsInformation[key]
    
    # get the unique ID for this document
    refDocID = document_references[doc["filename"]+doc["filesize"]]
    
    # sometimes the value is dict so retrieve the ID from that referenece  
    if str(type(refDocID)) == "<class 'dict'>":
        refDocID = refDocID["id"]
        
    if str(type(refDocID)) == "<class 'dict'>":
        refDocID = refDocID["id"]
    # add this to the document
    doc["unique id"] = refDocID
    
    # update the documentsInformation
    documentsInformation[key] = doc
    

100%|██████████| 52137/52137 [00:00<00:00, 323666.91it/s]


In [91]:
for key in tqdm(documentsInformation):
    doc = documentsInformation[key]
    if str(type(doc["unique id"])) == "<class 'dict'>":
        print(doc["unique id"])

100%|██████████| 52137/52137 [00:00<00:00, 852966.48it/s]


In [76]:
test == documentsInformation

True

In [71]:
# save the new documents and items info
with open("NewProcessingData/documentsInformation.txt", "wb") as fp:
    pickle.dump(documentsInformation, fp)
#with open("NewProcessingData/itemsInformation.txt", "wb") as fp:
#    pickle.dump(itemsInformation, fp)

In [53]:
# Read the new files
with open("NewProcessingData/itemsInformation.txt", "rb") as fp:   
    itemsInformation = pickle.load(fp)
with open("NewProcessingData/documentsInformation.txt", "rb") as fp:   
    documentsInformation = pickle.load(fp)

In [92]:
unique_docs_dir = "PDFs/UniqueDocuments"

downloaded_docs = [d + ".pdf" for d in os.listdir(unique_docs_dir)]

i=0
for key in tqdm(documentsInformation):
    doc = documentsInformation[key]
    
    # get the unique version of this doc
    uniqueDoc = documentsInformation[doc["unique id"]]
    uniqueDocID = uniqueDoc["document id"]
    #print(uniqueDocID)
    
    try: 
        # check whether this doc has already been downloaded
        if uniqueDocID + ".pdf" not in downloaded_docs:
            i+=1
    except:
        #print(str(type(uniqueDocID)) == "<class 'dict'>")
        i = i
i

100%|██████████| 52137/52137 [00:24<00:00, 2139.37it/s] 


28724

## Extracting matches from the items

In [54]:
# now we are going to loop through every item and 
# link a document to the item it is attached to

# Document ids will be used as keys and item ids will be appended to a list
document_item_match = {}

for key in tqdm(itemsInformation):
    item = itemsInformation[key]    
    docs = item["Documents"]
    
    # only look at the docs if there is at least one attached to this item
    if len(docs) >= 1:
        
        # now loop through every document attachted to the item
        for docID in docs:
            
            # get the reference to the unique ID of this document version
            docInfo = documentsInformation[docID["id"]]
            
            # search for the reference using the filename+filesize
            refDocID = document_references[docInfo["filename"]+docInfo["filesize"]]
            
            # sometimes the value is dict so retrieve the ID from that referenece        
            if refDocID is dict or str(type(refDocID)) == "<class 'dict'>":
                refDocID = refDocID["id"]
            
            # check if the key already exists in the dictionary.             
            if refDocID in document_item_match:
                document_item_match.setdefault(refDocID, []).append(key)
            # If not add it otherwise append item key
            else :
                document_item_match[refDocID] = [key]
                
# make a dataframe out of it with the item id as row and every column consisting of meeting ids            
document_item_match = pd.DataFrame.from_dict(document_item_match, orient = "index").reset_index()
document_item_match.columns = ["Document_id"] + ["item_id_"+ str(s) for s in document_item_match.columns[1:]]
document_item_match = document_item_match.set_index("Document_id")
#document_item_match.to_csv("NewProcessingData/document_item_match.csv", index=True)
document_item_match   

100%|██████████| 15314/15314 [00:00<00:00, 76886.19it/s]


,item_id_0,item_id_1,item_id_2,item_id_3,item_id_4,item_id_5,item_id_6,item_id_7,item_id_8,item_id_9,item_id_10,item_id_11,item_id_12
Document_id,,,,,,,,,,,,,
72cc4b22-0de8-4a83-ae9c-b4961b74050e,f0ba2c10-26ce-4441-8541-0c63ab65215f,None,None,None,None,None,None,None,None,None,None,None,None
501dca51-6764-47e5-9eff-9072ea71b758,dae7f992-a782-4c04-ac58-62c5272318cc,None,None,None,None,None,None,None,None,None,None,None,None
916ea22e-4016-45a2-ac43-2e0099706d72,be31f268-86db-42f8-985e-45d4a1147887,None,None,None,None,None,None,None,None,None,None,None,None
5ab63e8f-f46e-4495-98bd-cd486c1f6445,5dab871f-fd96-416f-ada3-0d17de8d9a9e,None,None,None,None,None,None,None,None,None,None,None,None
d7f0e1f5-c293-4c01-b805-89ae823e0639,5dab871f-fd96-416f-ada3-0d17de8d9a9e,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
fab436ca-85c5-40ff-8364-7014a7a86adc,7a56ee03-b9bb-4b92-9b5c-9d4920f6590a,None,None,None,None,None,None,None,None,None,None,None,None
7f00ca9d-6345-4433-96e1-ac7487228f0d,8c20e0fa-9227-4e77-90f5-1de4b96696f1,None,None,None,None,None,None,None,None,None,None,None,None
8d7f4434-af9e-4ca4-b0cb-1e0e70bec4e0,8c20e0fa-9227-4e77-90f5-1de4b96696f1,None,None,None,None,None,None,None,None,None,None,None,None


In [55]:
# count the total number of matches in the dataframe
total_matches = 0
for index, row in document_item_match.iloc[:,:].iterrows():
    for item in row:
        if item != None:
            total_matches+=1
total_matches

40706

# Testing the shortest paths

In [35]:
item_meeting_match = pd.read_csv("ProcessedMeetings/item_meeting_match.csv", index_col=0)
shortest_paths = pd.read_csv("Duplicate Detection/test_shortest_paths.csv").set_index("items")
shortest_paths

,f0607435-a601-4a06-85d4-63dd17ca3ec6,c6b49408-1158-495b-b69e-6137da9da1e7,0d47334d-7b3a-4b62-9ab6-7ff63fe49ebf,f804c38b-59f3-40c1-a9f3-c28c8b6078cc,400094ee-0e1f-432f-b1e4-ba41abdd516b,a9a3fed6-91dd-4885-9c0c-f4d9bad37ef1,2784c42c-760c-4d5a-9f45-2539604a8c6b,3c5a1fa5-2720-465a-b8ad-c8be74c5556a,e76d67a8-5f07-4e47-bbc5-29b1ed7ae7a0,b664cb20-7552-405e-957e-af7fec97f490,...,5b43d8ec-65b5-4d97-aee5-5050a921687c,76bc2df4-336a-4743-90ee-ba505a13ef8e,f46ad339-fecc-4228-9e48-b7ca8a83d93c,7a56ee03-b9bb-4b92-9b5c-9d4920f6590a,137f4d35-f4c8-497f-b149-b758276e2ecc,8c20e0fa-9227-4e77-90f5-1de4b96696f1,a6a6d28b-0715-419a-93ba-b95ce6ab6d50,9e63ca83-e8a7-48be-abfa-227b0f9e6116,8f199b1f-7e90-4007-b082-c8142e58ce35,7ef6e7b6-1082-4eb0-a150-46b5ec50e61d
items,,,,,,,,,,,,,,,,,,,,,
f0607435-a601-4a06-85d4-63dd17ca3ec6,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c6b49408-1158-495b-b69e-6137da9da1e7,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0d47334d-7b3a-4b62-9ab6-7ff63fe49ebf,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f804c38b-59f3-40c1-a9f3-c28c8b6078cc,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400094ee-0e1f-432f-b1e4-ba41abdd516b,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8c20e0fa-9227-4e77-90f5-1de4b96696f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
a6a6d28b-0715-419a-93ba-b95ce6ab6d50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
9e63ca83-e8a7-48be-abfa-227b0f9e6116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [36]:
def print_timeline(itemsList):
    # loop through every item in the list
    for item in itemsList:
        item_info = itemsInformation[item]
        print("Agenda Point:\t\t", item_info["features"], item_info["title"])
        print("Agenda point ID: \t", item_info["item id"])
        
        # find the meetinginfo for this docuemnt to show the data
        meetingID = item_meeting_match.loc[item].values        
        meeting = meetingsInformation[meetingID[0]]
        print("Agenda point date: \t", meeting["meetingdate"].split("T")[0])
        
        print("Attached documents:")
        
        docs = item_info["Documents"]
        if len(docs) > 0:
            for doc in docs:
                #print(doc)
                print("\t\t\t - ",documentsInformation[doc["id"]]["filename"])
        print("\n_________________________________________________________________________________________________________________")
        
# function for returning the most relevant items based on 
def return_related_items(df, id, max_distance, print_bool = False):
    Related_items = df.loc[id][df.loc[id] < max_distance].index
    
    if print_bool:
        print("The item given to this function:\n")    
        [print(x, itemsInformation[id][x]) for x in itemsInformation[id]]

        print("\nRelated item IDs:")
        [print(x) for x in Related_items]
    
    return Related_items
def find_item(itemName, itemsInformation, print_bool = False):
    for key in itemsInformation:
        
        if itemsInformation[key]["title"] == itemName:
            
            for child in itemsInformation[key]:
                if print_bool:
                    print(f"{child}: \t\t{itemsInformation[key][child]}")
            if print_bool:
                print("_________________________________________")
            return itemsInformation[key]["item id"]

    print("No item related to this name could be found")
    return ""

In [44]:
rel_items = return_related_items(shortest_paths, "cb25856c-ebd7-46df-b607-3b733e6d374a", 2, False)
print_timeline(rel_items)

Agenda Point:		 4 Bekrachtiging geheimhouding besluiten en bijlagen:   Hierbij verzoek ik u op grond van artikel 25, derde lid van de Gemeentewet de geheimhouding te bekrachtigen van de geheime bijlagen behorende bij agendapunten 11. en 30
Agenda point ID: 	 cb25856c-ebd7-46df-b607-3b733e6d374a
Agenda point date: 	 2006-06-29
Attached documents:
			 -  amendement 32.pdf
			 -  Amendement 39.pdf

_________________________________________________________________________________________________________________
Agenda Point:		 14 Debat Kandidatuur Utrecht voor Eurovisie Songfestival 2020
Agenda point ID: 	 ec4d6e8d-bb5c-4afe-9304-4973165f61af
Agenda point date: 	 2020-02-06
Attached documents:
			 -  Dossier 835 voorblad.pdf
			 -  Raadsbrief Kandidaatstelling Utrecht als gaststad Songfestival 2020.pdf
			 -  Bidbook Eurovisie Songfestival Utrecht 2020_geredigeerd.pdf
			 -  Raadsbrief Aanpassing geheimhouding bidbook Songfestival 2020.pdf
			 -  Raadsbrief Shortlist Eurovisie Songfestival

In [40]:
len(itemsInformation["c5dadb55-03a2-436c-bbff-7107d62eb1f9"]["Documents"])

130

In [25]:
shortest_paths.max().idxmax()

'cb25856c-ebd7-46df-b607-3b733e6d374a'